In [19]:
import cv2
import numpy as np
import torch
from sam2.sam2_image_predictor import SAM2ImagePredictor
from transformers import BlipProcessor, BlipForConditionalGeneration, CLIPProcessor, CLIPModel
from sentence_transformers import SentenceTransformer
import chromadb
import google.generativeai as genai
import os
from PIL import Image
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
# --- Carga de API keys desde .env ---
genai.configure(api_key=os.getenv('GEMINI_API_KEY'))

# --- Configuración de dispositivos ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Modelos para segmentación y caption ---
predictor = SAM2ImagePredictor.from_pretrained(
    "facebook/sam2-hiera-large",
    device="cpu"
)
processor_blip = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model_blip     = BlipForConditionalGeneration.from_pretrained(
    "Salesforce/blip-image-captioning-base"
).to(device)

# --- Modelos para RAG+CLIP ---
# CLIP para embeddings de crop
clip_model     = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
# MiniLM para embeddings de texto
model_text     = SentenceTransformer(
    "sentence-transformers/all-MiniLM-L6-v2",
    device=device
)

# --- Cliente y colecciones Chroma ---
client    = chromadb.PersistentClient(path="chroma")
text_col  = client.get_or_create_collection(
    name="tavascan_text",
    metadata={"hnsw:space": "cosine"}
)
img_col   = client.get_or_create_collection(
    name="tavascan_images",
    metadata={"hnsw:space": "cosine"}
)

# --- Lectura de imagen ---
img_bgr = cv2.imread("C:/Users/weiha/Documents/HackUPC_2025/img/dashboard.jpeg")
img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
predictor.set_image(img_rgb)

In [29]:
# Variables globales
ref_pt = []           # puntos de la selección (en coords de display)
drawing = False       # flag de arrastre
scale_percent = 100   # zoom inicial en porcentaje
img = img_bgr.copy()  # imagen original
h_orig, w_orig = img.shape[:2]

# Función para recalcular la imagen mostrada según el zoom
def update_display():
    global img_display, img_copy, scale
    scale = scale_percent / 100.0
    w_disp = int(w_orig * scale)
    h_disp = int(h_orig * scale)
    img_display = cv2.resize(img, (w_disp, h_disp), interpolation=cv2.INTER_AREA)
    img_copy = img_display.copy()
    # Si ya hay ref_pt, redibuja el rectángulo en img_copy
    if len(ref_pt) == 2:
        cv2.rectangle(img_copy, ref_pt[0], ref_pt[1], (0, 255, 0), 2)

# Callback para el trackbar de zoom
def on_trackbar(val):
    global scale_percent
    scale_percent = max(val, 1)  # evita 0%
    update_display()

# Callback para el mouse: dibuja y captura la caja en coords de display
def mouse_callback(event, x, y, flags, param):
    global ref_pt, drawing, img_copy
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ref_pt = [(x, y)]
    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        img_copy = img_display.copy()
        cv2.rectangle(img_copy, ref_pt[0], (x, y), (0, 255, 0), 2)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        ref_pt.append((x, y))
        img_copy = img_display.copy()
        cv2.rectangle(img_copy, ref_pt[0], ref_pt[1], (0, 255, 0), 2)
        # Mapea coords de display a coords originales
        x0_disp, y0_disp = ref_pt[0]
        x1_disp, y1_disp = ref_pt[1]
        x0 = int(x0_disp / scale)
        y0 = int(y0_disp / scale)
        x1 = int(x1_disp / scale)
        y1 = int(y1_disp / scale)
        print(f"Coordenadas en imagen original: ({x0},{y0}) → ({x1},{y1})")

# Prepara la ventana y los callbacks
update_display()
cv2.namedWindow("Selector de caja")
cv2.createTrackbar("Zoom %", "Selector de caja", scale_percent, 300, on_trackbar)
cv2.setMouseCallback("Selector de caja", mouse_callback)

# Bucle de visualización
while True:
    cv2.imshow("Selector de caja", img_copy)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("r"):       # 'r' resetea la selección
        ref_pt = []
        img_copy = img_display.copy()
    elif key == ord("q"):     # 'q' sale
        break

cv2.destroyAllWindows()

Coordenadas en imagen original: (483,107) → (672,186)


In [30]:
# tras el bucle y antes de usar predictor:
x0_disp, y0_disp = ref_pt[0]
x1_disp, y1_disp = ref_pt[1]

# Asegura que (x0_disp,y0_disp) sea la esquina superior-izquierda
x0_disp, x1_disp = sorted([x0_disp, x1_disp])
y0_disp, y1_disp = sorted([y0_disp, y1_disp])

# Mapea a coordenadas de la imagen original y clampa
x0 = int(x0_disp / scale)
y0 = int(y0_disp / scale)
x1 = int(x1_disp / scale)
y1 = int(y1_disp / scale)

# Evita salirse de los límites
x0 = max(0, min(x0, w_orig - 1))
y0 = max(0, min(y0, h_orig - 1))
x1 = max(1, min(x1, w_orig    ))
y1 = max(1, min(y1, h_orig    ))

box = np.array([[x0, y0, x1, y1]], dtype=np.int32)

with torch.inference_mode():
    masks, scores, logits = predictor.predict(
        box=box,
        multimask_output=False,
        return_logits=True
    )

mask = masks[0]
crop = img_rgb[y0:y1, x0:x1]

# convierte de RGB a BGR para cv2
crop_bgr = cv2.cvtColor(crop, cv2.COLOR_RGB2BGR)
cv2.imshow("Region seleccionada", crop_bgr)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [31]:
# --- RAG+CLIP retrieval ---
# 1) Embedding de la región con CLIP
inputs_img = clip_processor(images=crop, return_tensors="pt").to(device)
with torch.no_grad():
    img_emb = clip_model.get_image_features(**inputs_img)
    img_emb = img_emb / img_emb.norm(p=2, dim=-1, keepdim=True)
img_emb = img_emb.cpu().numpy()[0]
# 2) Query en colecciones
txt_hits = text_col.query(query_embeddings=[img_emb], n_results=4)
img_hits = img_col.query(query_embeddings=[img_emb], n_results=2)
# 3) Construir contexto
context_txt = "\n".join(
    f"- {doc} (p.{meta['page']}, col.{meta['column']})"
    for doc, meta in zip(txt_hits["documents"][0], txt_hits["metadatas"][0])
)
context_img = "\n".join(img_hits["documents"][0])

# 4) Prompt RAG
prompt = (
    "Eres un experto en el manual del CUPRA Tavascan.\n"
    "Información relevante del manual:\n" + context_txt + "\n"
    "Imágenes relacionadas:\n" + context_img + "\n"
    "¿Qué es la parte seleccionada en la imagen?"
)

# 5) Llamada a Gemini incluyendo la imagen
model = genai.GenerativeModel('gemini-2.0-flash-001')

# Convertir el crop a formato adecuado para Gemini
crop_pil = Image.fromarray(crop)  # Convertir de numpy array a PIL Image

# Crear consulta multimodal con texto e imagen
response = model.generate_content([prompt, crop_pil])
print("Respuesta RAG+CLIP:\n", response.candidates[0].content)

Respuesta RAG+CLIP:
 parts {
  text: "La parte seleccionada en la imagen parece ser la pantalla del Digital Cockpit del CUPRA Tavascan. Muestra información importante como la velocidad, indicaciones de navegación y advertencias.\n"
}
role: "model"

